In [1]:
from __future__ import print_function
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import tarfile
import csv
import gc
import tensorflow as tf

from IPython.display import display, Image
from scipy import ndimage
from sklearn.linear_model import LogisticRegression
from six.moves.urllib.request import urlretrieve
from six.moves import cPickle as pickle
from IPython.display import display, Image

from os import listdir
from os.path import isfile, join
import random
from PIL import Image

# Config the matplotlib backend as plotting inline in IPython
%matplotlib inline

In [ ]:


Prelucrare imagini:
    
    

In [10]:
mypath = "images_test_rev1/"
newPath = "test_small3/"
files = listdir(mypath)
newFiles = listdir(newPath)

for id in range(len(files)):
    path = mypath + str(files[id])
    try:
        img2 = Image.open(path).convert('L');  #greyscale
        half_the_width = img.size[0] / 2
        half_the_height = img.size[1] / 2
        img3 = img2.crop(
            (
                half_the_width - 100,
                half_the_height - 100,
                half_the_width + 100,
                half_the_height + 100
            )
        )     
        img4 = img3.resize([80,80]) 
        file_name = str(files[id]);
        img4.save(newPath + file_name) #salvează imaginea 
        if (id%500 == 0):
            gc.collect()
    except IOError as e:
      print('Could not read:', file_name, ':', e, '- it\'s ok, skipping.')


In [11]:
mypath = "images_test_rev1/"
newPath = "test_small3/"
files = listdir(mypath)
print(len(files))
newFiles = listdir(newPath)
print(len(newFiles))

79975
79975


In [55]:
image_size = 80  # Pixel width and height.
pixel_depth = 255.0  # Number of levels per pixel.
test_folder = '.\\test_small3'

def load_images(folder, min_num_images):
  """Load the data for a single  folder."""

  image_files = os.listdir(folder)
  dataset = np.ndarray(shape=(len(image_files), image_size, image_size),
                         dtype=np.float32)
  ids = np.ndarray(shape=(len(image_files)), dtype=(np.str_, 6))
  print(folder)
  num_images = 0

  for image in image_files:
    image_file = os.path.join(folder, image)
    try:
      image_data = (ndimage.imread(image_file).astype(float) - 
                    pixel_depth / 2) / pixel_depth
      if image_data.shape != (image_size, image_size):
        raise Exception('Unexpected image shape: %s' % str(image_data.shape))
      dataset[num_images, :, :] = image_data
      ids[num_images] = image
      #print(ids[num_images])
      num_images = num_images + 1
    except IOError as e:
      print('Could not read:', image_file, ':', e, '- it\'s ok, skipping.')
    
  dataset = dataset[0:num_images, :, :]
  ids = ids[0:num_images]
  if num_images < min_num_images:
    raise Exception('Many fewer images than expected: %d < %d' %
                    (num_images, min_num_images))
    
  print('Full dataset tensor:', dataset.shape)
  print('Mean:', np.mean(dataset))
  print('Standard deviation:', np.std(dataset))
  return dataset, ids
        
def maybe_pickle(folder, min_num_images_per_class, force=False):
    dataset_names = []

    print(folder)

    set_filename = folder + '1.pickle'
    dataset_names.append(set_filename)
    if os.path.exists(set_filename) and not force:
      #override by setting force=True.
      print('%s already present - Skipping pickling.' % set_filename)
    else:
      print('Pickling %s.' % set_filename)
      dataset, ids = load_images(folder, min_num_images_per_class)
      try:
        with open(set_filename, 'wb') as f:
          save = {
            'dataset': dataset,
            'ids' : ids
            }
          pickle.dump(save, f, pickle.HIGHEST_PROTOCOL)
      except Exception as e:
        print('Unable to save data to', set_filename, ':', e)
  
    return dataset_names
  

train_datasets = maybe_pickle(test_folder, 79975)

.\test_small3
Pickling .\test_small31.pickle.
.\test_small3
Full dataset tensor: (79975, 80, 80)
Mean: -0.405905
Standard deviation: 0.128175


In [56]:
gc.collect()

0



Generare CSV cu rezultate:




In [2]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range
from math import sqrt
import matplotlib.pyplot as plt
import gc
import csv

In [3]:
image_size = 80
num_labels = 37
num_channels = 1 # grayscale

def reformat(dataset):
  dataset = dataset.reshape((-1, image_size, image_size, num_channels)).astype(np.float32)
  return dataset

def accuracy(predictions, labels):
    return np.sqrt(((predictions - labels) ** 2).mean())


In [4]:
pickle_file = 'test_small31.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  test_dataset = (save['dataset'])
  test_id = (save['ids'])

  del save  # hint to help gc free up memory
    
  print('Test set', test_dataset.shape, test_id.shape)
  


Test set (79975, 80, 80) (79975,)


In [5]:


test_dataset = reformat(test_dataset)

print('Test set', test_dataset.shape, test_id.shape)

Test set (79975, 80, 80, 1) (79975,)


In [6]:
#   MODEL 5
batch_size = 128 #109
patch_size = 5
depth = 32 #32, 16
num_hidden = 64 #100
num_hidden2 = 64 #37
beta = 0.0002#0.0002
graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=sqrt(2.0/(image_size // 4 * image_size // 4 * depth * num_hidden))))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  print(str(num_hidden))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_hidden2], stddev=sqrt(2.0/(num_hidden * num_hidden2))))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden2]))
  print(str(num_hidden2))
  layer5_weights = tf.Variable(tf.truncated_normal(
      [num_hidden2, num_labels], stddev=sqrt(2.0/(num_hidden2 * num_labels))))
  layer5_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))

  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    hidden1 = tf.nn.relu(conv + layer1_biases)
    hidden =  tf.nn.max_pool(hidden1, [1,2,2,1], [1,2,2,1],padding='SAME')
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 1, 1, 1], padding='SAME')
    hidden1 = tf.nn.relu(conv + layer2_biases)
    hidden =  tf.nn.max_pool(hidden1, [1,2,2,1], [1,2,2,1],padding='SAME')
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    hidden2 = tf.nn.relu(tf.matmul(hidden, layer4_weights) + layer4_biases)
    return tf.matmul(hidden2, layer5_weights) + layer5_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss1 = tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(logits, tf_train_labels))))
  loss = loss1 + beta * (tf.nn.l2_loss(layer1_weights) + tf.nn.l2_loss(layer2_weights) + tf.nn.l2_loss(layer3_weights) + tf.nn.l2_loss(layer4_weights) + tf.nn.l2_loss(layer5_weights))
    
 # Optimizer.
  global_step = tf.Variable(0)  # count the number of steps taken.
  #learning_rate = tf.train.exponential_decay(0.3, global_step, 5000 ,0.96, staircase = True) #5000
  learning_rate = tf.Variable(0.3)
  optimizer = tf.train.AdagradOptimizer(learning_rate).minimize(loss, global_step=global_step)
   

  # Predictions for the training, validation, and test data.
  def prediction(logits):
    pred1 = tf.nn.softmax(logits[:,0:3])
    pred2 = tf.nn.relu(logits[:,3:num_labels])
    pred = tf.concat([pred1, pred2],1)
    return pred
  train_prediction1 = prediction(logits)
   #drop out some of the predictions
  train_prediction = tf.nn.dropout(train_prediction1,0.5)
  


  

  tf.add_to_collection('vars',  layer1_weights)
  tf.add_to_collection('vars',  layer2_weights)
  tf.add_to_collection('vars',  layer3_weights)
  tf.add_to_collection('vars',  layer4_weights)
  tf.add_to_collection('vars',  layer5_weights)
  tf.add_to_collection('vars',  layer1_biases)
  tf.add_to_collection('vars',  layer2_biases)
  tf.add_to_collection('vars',  layer3_biases)
  tf.add_to_collection('vars',  layer4_biases)
  tf.add_to_collection('vars',  layer5_biases)
  saver = tf.train.Saver(write_version=tf.train.SaverDef.V2)

64
64


In [8]:
gc.collect()

results = np.ndarray(shape=(test_id.shape[0],38))
acc = 0
t = []
#with open(filename, 'w') as myfile:
    #wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()

    saver.restore(session, './my-model5.ckpt')
    print("Model restored.") 
    print('Initialized')
    for i in range(0,79):
        results[i*1000:(i+1)*1000,0] = test_id[i*1000:(i+1)*1000]
        results[i*1000:(i+1)*1000,1:38] = prediction(model(test_dataset[i*1000:(i+1)*1000,:,:,:])).eval()
        #for j in range(0,37):
         #   a.append(pre[0,j])
        
        gc.collect()
        print(i)
            #wr.writerow(a)
    results[79*1000:test_id.shape[0],0] = test_id[79*1000:test_id.shape[0]]
    results[79*1000:test_id.shape[0],1:38] = prediction(model(test_dataset[79*1000:test_id.shape[0],:,:,:])).eval()
    np.savetxt("results2.csv", results, delimiter=",")
    

Model restored.
Initialized
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
